# Statistics - Models
Date : 10/08/2020

Load the csv gathering the models and their results.
The aim is to show the evolution of the accuracy and the parameters

In [1]:
import torch

In [2]:
# useful imports
from IPython.display import SVG, Audio, display
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
import librosa
%matplotlib inline
import os
from matplotlib import pyplot as plt

# voicemap imports
from voicemap.datasets import LibriSpeech, SpeakersInTheWild, CommonVoice, TCOF, ClassConcatDataset, SpectrogramDataset, DatasetFolder 
from config import PATH, DATA_PATH, LOG_PATH

## (Optional) Save a model

In [5]:
old_model_name = "../models/model=resnet__dim=1__lr=1e-06__weight_decay=0.01__momentum=0.9__filters=128__batch_size=1500__n_seconds=3.0__use_standardized=True__spectrogram=True__n_t=0__T=None__n_f=0__F=None__num_samples=45255__num_classes=3017.pt"
new_model_name = "std_000001.pt"

In [ ]:
%%bash -s "$old_model_name" "$new_model_name"
cp $1 ../saved_models/$2

## (Optional) Append the model into the 'models' log file

In [ ]:
def appendModel(old_name, new_name="Untitled"):
    old_name = old_name.split("/")[-1][:-3]
    df = pd.read_csv(LOG_PATH + '/models.csv')
    params = old_name.split("__")
    i = len(df)
    df.loc[i] = np.NaN
    df.loc[i, 'model_name'] = new_name
    for param in params:
        param_name, param_value = param.split("=")
        df.loc[i, param_name] = param_value
    return df
df = appendModel(old_model_name, new_model_name)
df.to_csv(f"{LOG_PATH}/models.csv", index=False)

## Load data

In [3]:
df_read = pd.read_csv(f"{LOG_PATH}/models.csv")
df_read

,model_name,details,model,dim,lr,weight_decay,momentum,filters,batch_size,n_seconds,...,window_hop,n_t,T,n_f,F,num_samples,num_classes,datasets,val_loss,accuracy
0,gaixoa.pt,NaN,resnet,1.0,0.000100,0.01,0.9,128.0,32.0,3.0,...,NaN,0.0,None,0.0,None,130475,1291,LS,0.1200,0.6580
1,babybabyMario.pt,NaN,resnet,1.0,0.000100,0.01,0.9,128.0,1500.0,3.0,...,NaN,0.0,None,0.0,None,133376,1291,LS,0.2380,0.7300
2,luigi_000001.pt,with commonvoice,resnet,1.0,0.000001,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,263978,3045,LSC,0.4136,0.8437
3,peach_00001.pt,with TCOF,resnet,1.0,0.000010,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,465597,3699,LSCT,1.0800,0.5800
4,std_00001.pt,with 15-standardized,resnet,1.0,0.000010,0.01,0.9,128.0,1500.0,3.0,...,0.01,0.0,None,0.0,None,45255,3017,LCT,1.2108,0.2200


## 1. Evolution of accuracy & loss

Avant l'ajout de TCOF (et donc des voix d'enfants) on obtient un excellent résultat. Le modèle n'était cependant pas performant avec des voix d'enfants.
L'ajout de TCOF a fait largement augmenté la perte et baissé la précision globale. L'amélioration sur les locuteurs enfants n'a pas encore été testée.

#### Affichage des logs des entraînements des modèles

In [7]:
model_log = pd.read_csv("../logs/model=resnet__dim=1__lr=1e-05__weight_decay=0.01__momentum=0.9__filters=128__batch_size=1500__n_seconds=3.0__spectrogram=True__n_t=0__T=None__n_f=0__F=None__num_samples=465597__num_classes=3699.csv")
model_log

,epoch,accuracy,common_voice_test_auc,common_voice_test_eer,librispeech_dev_clean_auc,librispeech_dev_clean_eer,loss,lr,sitw_eval_auc,sitw_eval_eer,tcof_dev_auc,tcof_dev_eer,train_accuracy,train_loss,val_accuracy,val_loss
0,1,0.580135,0.995919,0.004081,0.951533,0.103746,3.424099,0.00001,0.671032,0.428238,0.998809,0.006651,0.919583,1.097484,0.922401,1.088119
1,2,0.582177,0.995199,0.004801,0.951195,0.103746,3.404816,0.00001,0.667161,0.425987,0.998775,0.006892,0.919333,1.089387,0.922251,1.089520
2,3,0.582929,0.995919,0.004081,0.951713,0.103746,3.396156,0.00001,0.668838,0.409745,0.998878,0.006651,0.923083,1.084272,0.921993,1.089486
3,4,0.583907,0.995679,0.004321,0.952446,0.103746,3.386068,0.00001,0.671181,0.428571,0.998775,0.007132,0.921375,1.085931,0.922165,1.090254
4,5,0.584557,0.996159,0.003841,0.951660,0.103746,3.376809,0.00001,0.669037,0.423977,0.998775,0.007372,0.924875,1.065528,0.921735,1.090586
5,6,0.585256,0.996639,0.003361,0.952165,0.103746,3.371813,0.00001,0.670139,0.428571,0.998775,0.007372,0.923875,1.083947,0.922358,1.091296
6,7,0.585932,0.995439,0.004561,0.951427,0.103746,3.362241,0.00001,0.675438,0.428571,0.998878,0.006411,0.922042,1.096264,0.922616,1.092424
